In [1]:
!python3 -m spacy download en_core_web_sm

2022-10-09 09:54:57.681942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import json
import string
import spacy
import re
from gensim.models import Word2Vec
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words

/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-09 18:18:19.172055: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
test_data_file = './cuad-data/test_contract.json'
contract_terms_file = './cuad-data/contract_terms.json'

def prep_data():
    with open(test_data_file) as json_file:
        data = json.load(json_file)
    contract = data['contract']
    return contract

def load_terms():
    with open(contract_terms_file) as json_file:
        data = json.load(json_file)
    terms = data['terms']
    return terms

article_text = prep_data() 

In [4]:
class PreProcessText(object):

    def __init__(self):
        pass

    def __remove_punctuation(self, text):
        """
        Takes a String
        return : Return a String
        """
        message = []
        for x in text:
            if x in string.punctuation:
                pass
            else:
                message.append(x)
        message = ''.join(message)

        return message

    def __remove_stopwords(self, text):
        """
        Takes a String
        return List
        """
        words= []
        for x in text.split():

            if x.lower() in stopwords:
                pass
            else:
                words.append(x)
        return words


    def token_words(self,text=''):
        """
        Takes String
        Return Token also called  list of words that is used to
        Train the Model
        """
        message = self.__remove_punctuation(text)
        words = self.__remove_stopwords(message)
        return words

In [5]:
print("Downloaded Stop words ...... ")
helper = PreProcessText()
#words = helper.token_words(text=txt)
words = helper.token_words(text=article_text)

Downloaded Stop words ...... 


In [6]:
model = Word2Vec([words], min_count=1)
#model = Word2Vec([words], size=100, window=5, min_count=1, workers=4)

In [14]:
catchwords = load_terms() 
catch_stmt = {}
return_value = {}
sim_words =[]

print("Contract: \n\"", article_text, "\" \n")
print("Catch Words: ", catchwords, "\n")
for c_word in catchwords: 
    print(">>>> Searching Word: \"", c_word, "\"" )
    try : 
        sim_words = model.wv.most_similar(c_word)
        print (">>>>  Similar Words 1: \"", sim_words)
    except KeyError: # TODO: Why this is thrown ?? 
        print (">>>> Words Similar to \"" + c_word + "\" not found.")
    print (">>>>  Similar Words 2: \"", sim_words)
    for s_word in sim_words:
        print (">>>>>>>> Similar word \"" , s_word[0] , "\" of \"", c_word, "\" Searching ...")
        catch_stmt[s_word[0]] = [sentence + '.' for sentence in article_text.split('.') if s_word[0] in sentence]
        try: 
            print (">>>>>>>>  Found Sentence : ", catch_stmt[s_word[0]][0])
            res = re.search(catch_stmt[s_word[0]][0], article_text)
            if res:
                print(">>>>>>>> Index : ", res.start(), res.end())
                print(">>>>>>>>  Output String : ", article_text[res.start() : res.end()])   
                stmt_index = str(res.start()) + "-" + str(res.end())
                if stmt_index in return_value.keys():                           
                    relevence = return_value[stmt_index]["relevence"] + 10
                else:    
                    relevence = 10
                return_value[stmt_index] = {"start_index" : res.start(), "end_index" : res.end(), "relevence" : relevence}
        except IndexError:
            print (">>>>>>>> Word Not Found")
print ("return_value : \n", return_value)

high_score = 0 
for r_key in return_value:
    if high_score < return_value[r_key]['relevence']:
        high_score = return_value[r_key]['relevence']
#print ("Highest Score:", high_score)
for r_key in return_value:
    score = 100 * return_value[r_key]['relevence'] / high_score 
    #print(return_value[r_key]['relevence'] , " >> " , score)
    if score > 67: 
        return_value[r_key]["relevence_degree"] = "HIGH"
    else: 
        if score > 34: 
            return_value[r_key]["relevence_degree"] = "MEDIUM"
        else:
            if score > 0: 
                return_value[r_key]["relevence_degree"] = "LOW"

print ("return_value : \n", return_value)

Contract: 
" Broadcasting, Licensing and Wireless are referred to collectively herein as the "Buyers." Liability are very high. Subject to the terms and conditions of this Agreement, the Buyers hereby agree to purchase substantially all of the assets (and assume certain of the liabilities) of the Seller that are used or useful in the operation of radio stations KNRQ-FM (licensed to Creswel, Oregon), and KNRQ-AM and KZEL-FM (licensed to Eugene, Oregon) (the "Stations") in return for cash. Now, therefore, in consideration of the above premises and the mutual promises herein made, and in consideration of the representations, warranties, and covenants herein contained, the Parties agree as follows. Business is showing in low legal compliance. On and subject to the terms and conditions of this Agreement, the Seller agrees to sell, transfer, convey and deliver to (i) Licensing, and Licensing agrees to purchase from the Seller, all of the FCC Licenses listed in Section 2(k) of the disclosure 

In [ ]:
import app.qna_service as qna_service
import app.ner_service as ner_service 

contract = ner_service.prep_data()
nlp = ner_service.initialise()
for k in catch_stmt.keys():
    for stmt in catch_stmt.get(k):
        print (">>> Statement >>> " + stmt)
        ner_result = ner_service.predict(stmt, nlp)
        print (">>> NER >>> ")
        print(ner_result)

>>> Statement >>>   (708) 598-4671                   Attn:  Jim Stumpe

                  With a copy to:                   Thomas V.
>>> NER >>> 
[{'entity': 'B-PER', 'score': 0.99969816, 'index': 15, 'word': 'Jim', 'start': 42, 'end': 45}, {'entity': 'I-PER', 'score': 0.9997372, 'index': 16, 'word': 'St', 'start': 46, 'end': 48}, {'entity': 'I-PER', 'score': 0.9995466, 'index': 17, 'word': '##ump', 'start': 48, 'end': 51}, {'entity': 'I-PER', 'score': 0.66749567, 'index': 18, 'word': '##e', 'start': 51, 'end': 52}, {'entity': 'B-PER', 'score': 0.9986609, 'index': 24, 'word': 'Thomas', 'start': 106, 'end': 112}, {'entity': 'I-PER', 'score': 0.99531734, 'index': 25, 'word': 'V', 'start': 113, 'end': 114}]
>>> Statement >>>   Company shall at all times make reasonable  efforts                   to  maintain  quality  control  and  to  deliver  Products  to                   Distributor  which,  when received by Distributor,  or, as the                   case  may  be,  the  end-user,  a